In [184]:
import numpy as np
from numpy import linalg as LA
from scipy.sparse import linalg as SLA
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 7.5 ms


## 1-a

In [42]:
x_2016 = np.genfromtxt('/Users/Amiros/GitHub/Machine Learning for Data Science/hw5_markov_chaines/CFB2016_scores.csv', 
          delimiter=',')

team_names = pd.read_csv('/Users/Amiros/GitHub/Machine Learning for Data Science/hw5_markov_chaines/TeamNames.txt',
                        header = None)

time: 63.9 ms


In [71]:
a_index = x_2016[:,0] -1
b_index = x_2016[:,2] -1

a_score = x_2016[:,1]
b_score = x_2016[:,3]

a_indicator = np.where(a_score>b_score,1,0)
b_indicator = np.where(a_score<b_score,1,0)

time: 7.95 ms


In [237]:
M = np.zeros((760, 760))

M[tuple(a_index), tuple(a_index)] = M[tuple(a_index), tuple(a_index)] + a_indicator + a_score/(a_score+b_score)
M[tuple(b_index), tuple(b_index)] = M[tuple(b_index), tuple(b_index)] + b_indicator + b_score/(a_score+b_score)
M[tuple(a_index), tuple(b_index)] = M[tuple(a_index), tuple(b_index)] + b_indicator + b_score/(a_score+b_score)
M[tuple(b_index), tuple(a_index)] = M[tuple(b_index), tuple(a_index)] + a_indicator + b_score/(a_score+b_score)

M_normalize = M/M.sum(axis=1)

time: 37.9 ms


/Users/Amiros/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:3: VisibleDeprecationWarning: non integer (and non boolean) array-likes will not be accepted as indices in the future
  app.launch_new_instance()
/Users/Amiros/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:4: VisibleDeprecationWarning: non integer (and non boolean) array-likes will not be accepted as indices in the future
/Users/Amiros/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:5: VisibleDeprecationWarning: non integer (and non boolean) array-likes will not be accepted as indices in the future
/Users/Amiros/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:6: VisibleDeprecationWarning: non integer (and non boolean) array-likes will not be accepted as indices in the future


In [189]:
# function to calculate state at t
def update_w(w_0, t):
    w_arr = []
    w = w_0
    for i in range(t):
        w = np.dot(w, M_normalize)
    w_arr = pd.DataFrame(w.T, index= None, columns = ['score'])
    w_arr['team'] = team_names
    return w_arr

time: 7.38 ms


In [190]:
np.random.seed(42)
w_0 = np.random.uniform(low=0.0, high=1.0, size=(1,760))

w_10 = update_w(w_0, 10)
w_100 = update_w(w_0, 100)
w_1000 = update_w(w_0, 1000)
w_10000 = update_w(w_0, 10000)

time: 4.23 s


In [191]:
w_10000.sort('score', ascending=False).head(25)

/Users/Amiros/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,score,team
133,3.821612,Clemson
5,3.423369,Alabama
440,3.149235,NW Missouri St
219,2.674116,Florida St
581,2.363243,Southern Cal
751,2.269488,Wisconsin
704,2.264506,Washington
506,2.211617,Penn State
492,2.185432,Oklahoma
215,2.143253,Florida


time: 18 ms


## 1-b

In [222]:
u = SLA.eigs(M,1, sigma=1)[1].T
w_stationary = u/np.sum(u)

time: 91.7 ms


In [239]:
def print_w(w_0, w_stationary, t=2):
    w = w_0
    x = []
    for i in range(t):
        w = np.dot(w, M_normalize)
        x.append(LA.norm(w_stationary-w, ord=1))
    
    plt.figure(num=None, figsize=(10, 10), dpi=80)
    plt.plot(x, linewidth=2)

print_w(w_0, w_stationary, t=10000)

time: 32.2 ms


## 2-a